# 최근 7일간 코로나 현황 데이터 크롤링

In [79]:
from selenium import webdriver

In [80]:
driver = webdriver.Chrome('C:/playwithdata/chromedriver.exe')

url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%BD%94%EB%A1%9C%EB%82%98+%ED%98%84%ED%99%A9'
driver.get(url)

In [81]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [82]:
data = driver.find_elements_by_css_selector('div.column._column')

In [83]:
len(data) #최근 일주일 데이터

7

# 최근 일주일 데이터를 클릭하면서 리스트에 저장

In [84]:
dates = []
today_total = []
local = []
oversea = []

In [85]:
for today in data:
    today.click() #클릭
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2,'html.parser')
    
    #데이터로 접근 
    data2 = soup2.select('div.tooltip_area._tooltip_wrapper > dl > div') 
    
    dates.append(data2[0].select('dd')[0].text)
    today_total.append(data2[0].select('dd')[1].text)
    local.append(data2[1].select('dd')[0].text)
    oversea.append(data2[1].select('dd')[1].text)

In [6]:
# , 문자 제거해서 int형으로 만들 수 있게 만들기
for i in range(len(today_total)):
    today_total[i] = today_total[i].replace(',',"")    
    local[i] = local[i].replace(',',"")
    oversea[i] = oversea[i].replace(',',"")

In [8]:
# int형으로 만들기
today_total = list(map(int, today_total))
local = list(map(int, local))
oversea = list(map(int, oversea))

print(today_total)
print(local)
print(oversea)

[2050, 1877, 1626, 1417, 1507, 2154, 1882]
[1999, 1811, 1588, 1369, 1468, 2113, 1829]
[51, 66, 38, 48, 39, 41, 53]


In [68]:
print(dates)
print(today_total)
print(local)
print(oversea)

['8.19', '8.20', '8.21', '8.22', '8.23', '8.24', '8.25']
[2050, 1877, 1626, 1417, 1507, 2154, 1882]
[1999, 1811, 1588, 1369, 1468, 2113, 1829]
[51, 66, 38, 48, 39, 41, 53]


# DF 엑셀로 저장하기 

In [9]:
import pandas as pd

In [52]:
dic = {
    '날짜' : dates,
    '신규합계' : today_total,
    '국내발생' : local,
    '해외유입' : oversea
}

In [59]:
df = pd.DataFrame(dic)

In [60]:
df

,날짜,신규합계,국내발생,해외유입
0,8.19,2050,1999,51
1,8.20,1877,1811,66
2,8.21,1626,1588,38
3,8.22,1417,1369,48
4,8.23,1507,1468,39
5,8.24,2154,2113,41
6,8.25,1882,1829,53


In [61]:
df['날짜'] = df['날짜'].apply(lambda x: '2021-' + x.replace('.','-'))

In [62]:
df

,날짜,신규합계,국내발생,해외유입
0,2021-8-19,2050,1999,51
1,2021-8-20,1877,1811,66
2,2021-8-21,1626,1588,38
3,2021-8-22,1417,1369,48
4,2021-8-23,1507,1468,39
5,2021-8-24,2154,2113,41
6,2021-8-25,1882,1829,53


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      7 non-null      object
 1   신규합계    7 non-null      int64 
 2   국내발생    7 non-null      int64 
 3   해외유입    7 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 352.0+ bytes


In [64]:
df.to_excel('./making/covid_7days.xlsx',index = False)

# DF 데이터베이스에 저장하기

In [65]:
import pymysql

conn = pymysql.connect(host= '127.0.0.1', 
                        user = 'root', 
                        password='1234', 
                        db = 'project1', 
                        port=3307, 
                        charset='utf8')

cur = conn.cursor()

In [66]:
cur.execute("CREATE TABLE IF NOT EXISTS covid_7days (date date, new int, fromin int, fromout int)")

0

In [67]:
sql = 'insert into covid_7days (date, new, fromin, fromout) values(%s, %s, %s, %s)'

for idx in range(len(df)):
	cur.execute(sql, tuple(df.values[idx]))
    
conn.commit()

conn.close()